In [448]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os 
import csv#For making directories

# <font color="red">Exercise 1</font>
<font color="red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</font>

For this exercise I scrape off data from the website of one of the largest newspapers in northern India - The Tribune. I scrape off data of matrimonial ads, which in India, are published in newspapers by people seeking prospective brides and grooms. This paper is published in all major states of northern India, and for exercise 1, I scrape off the matrimonial ads for 'brides' and 'grooms' for 25th october 1998 and store them in a data frame.

In [449]:
tribune_request = requests.get('https://www.tribuneindia.com/1998/98oct25/class.htm')

tribune_content_Soup = bs4.BeautifulSoup(tribune_request.text, 'html.parser')


In [454]:
contentPTags = tribune_content_Soup.body.findAll('tr')

bride_info = []
groom_info = []
for trTag in contentPTags:
    for tdTag in trTag.findAll('td'):
        if tdTag.text == 'Brides Wanted':
            for pTag in trTag.findAll('p'):
                new_s = re.sub(r'[\n,".—-]','',pTag.text) 
                new_s1 = (re.sub(r'\r  {2,}|\bBox|\bTribune|\b([A-Z]*\d[A-Z]*){5,}\b|\bChandigarh|\bPh|\b-OL|\s+$'\
                                        ,' ',new_s))
                bride_info.append(re.sub(r'\(\d+\)|\s*$','',new_s1))
                with open(file, 'a') as f:
                    writer = csv.writer(f, delimiter = ',')
                    writer.writerow([re.sub(r'\(\d+\)|\s*$','',new_s1)] )
         
                
        if tdTag.text == 'Grooms Wanted':
            for pTag in trTag.findAll('p'):
                new_s = re.sub(r'[\n,.—-]','',pTag.text) 
                new_s1 = (re.sub(r'\r  {2,}|\bBox|\bTribune|\b([A-Z]*\d[A-Z]*){5,}\b|\bChandigarh|\bPh|\b-OL|\s+$'\
                                        ,' ',new_s))
                groom_info.append(re.sub(r'"\(\d+\)|\s*$','',new_s1))
                

In [446]:
matrimonials = pandas.DataFrame({'Brides-wanted' : pandas.Series(bride_info), 'Grooms-wanted':pandas.Series(groom_info)})
matrimonials

,Brides-wanted,Grooms-wanted
0,Alliance invited by well established and resp...,Well settled industrialist businessman match f...
1,Match for Delhi based Ramgarhia Sikh boy 28/6'...,Match for Punjabi Sarswat Brahmin slim girl 23...
2,MBBS bride for Hindu Arora educated boy 25 5'7...,Vegetarian professional match for Jat Sikh bea...
3,Professionally qualified beautiful girl prefer...,"Suitable match for 5'2""/27 convent educated ..."
4,USA/Canada citizen match for very handsome tee...,Professionals MCAs clean shaven match for well...
5,Highly cultured beautiful wellqualified girl f...,Large 4 Star Hotel with International collabo...
6,US Citizen Jat Sikh parents seek MBBS MBA elec...,NaN
7,Graceful match for young handsomeSikh clean sh...,NaN



# <font color="red">Exercise 2</font>
<font color="red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</font>



For exercise 2, I scrape off ads from the website of the newspaper for the years 2002, and 2014, and put them in sepateare dataframes with 'brides-wanted' and 'grooms-wanted' as columns. I also save the data into csv files for the purpose of reusing it if need be.

In [455]:

tribune_url = 'https://www.tribuneindia.com/archive'
base_webpage_content = requests.get(tribune_url)
web_page_content_soup = bs4.BeautifulSoup(base_webpage_content.text, 'html.parser')
base_url = 'https://www.tribuneindia.com'


tagLinks = web_page_content_soup.findAll('a', href=re.compile('/2014/form'), class_=False)
final_urls_1998 = set()
for aTag in tagLinks[:2]:
        relurl = aTag.get('href')
        level_1_url = base_url+relurl 
        level_1_request = requests.get(level_1_url)
        level_1_soup = bs4.BeautifulSoup(level_1_request.text, 'html.parser')
        level_1_tagLinks = level_1_soup.body.findAll('a',href = re.compile(aTag.text), class_=False)
       
        for a1Tag in level_1_tagLinks:
            level_2_url = a1Tag.get('href')
            level_2_request = requests.get(level_2_url)
            level_2_soup = bs4.BeautifulSoup(level_2_request.text, 'html.parser')
            level_2_tagLinks = level_2_soup.body.findAll('a',href = re.compile('class.htm'), class_=False)
            for a2Tag in level_2_tagLinks:
                final_urls_1998.add(base_url+'/'+aTag.text+re.sub(r'^..','',a2Tag.get('href')))
                
        break
                

In [532]:
def getTextFromTribunePage(targetURL, file_1, file_2):
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    lst_1 = []
    lst_2 = []
    contentPTags = soup.findAll('p')
    
   
    if 'class-f' in targetURL:
         for ptag in contentPTags:
          
            new_s = re.sub(r'[\n,".—-]','',ptag.text) 
            new_s1 = (re.sub(r'\r  {2,}|\bBox|\bTribune|\b([A-Z]*\d[A-Z]*){5,}\b|\bChandigarh|\bPh|\b-OL|\s+$'\
                                        ,' ',new_s))
                
            lst_2.append(re.sub(r'\(\d+\)|\s*$', '', new_s1))
            with open(file_1, 'a') as f:
                writer = csv.writer(f)
                writer.writerow([re.sub(r'\(\d+\)|\s*$', '', new_s1)] )
    if 'class-m' in targetURL:
        for ptag in contentPTags:
            new_s = re.sub(r'[\n,".—-]','',ptag.text) 
            new_s1 = (re.sub(r'\r  {2,}|\bBox|\bTribune|\b([A-Z]*\d[A-Z]*){5,}\b|\bChandigarh|\bPh|\b-OL|\s+$'\
                                        ,' ',new_s))
                
          
            lst_1.append(re.sub(r'\(\d+\)|\s*$', '', new_s1))
            with open(file_2, 'a') as f:
                writer = csv.writer(f)
                writer.writerow([re.sub(r'\(\d+\)|\s*$', '', new_s1)] )
        
                
    return pandas.DataFrame({'Brides-wanted' : pandas.Series(lst_1), 'Grooms-wanted':pandas.Series(lst_2)})


In [465]:
#Starting with level 0
base_url = 'https://www.tribuneindia.com/2014/forms/archive2.htm'

url_request = requests.get(base_url)
web_page_soup = bs4.BeautifulSoup(url_request.text,'html.parser')
tagLinks = web_page_soup.findAll('a', href=re.compile('2002'), class_=False)
final_urls_2002 = set()

# getting all relevant links and this is the next level
for aTag in tagLinks:
        relurl = aTag.get('href')
        level_1_request = requests.get(relurl)
        level_1_soup = bs4.BeautifulSoup(level_1_request.text, 'html.parser')
        level_1_tagLinks = level_1_soup.findAll('a',href = re.compile('class'), class_=False)
        
        # going further down another level and getting all relevant links for the next level 
        # and creating final set of urls
        for a1Tag in level_1_tagLinks:
            relurl = a1Tag.get('href')
            level_2_url = 'https://www.tribuneindia.com/2002'+re.sub(r'^..','',relurl)
            final_urls_2002.add(level_2_url)
       

In [533]:
# getting all info for 2002 and storing in dataframe
final_df_2002 = pandas.DataFrame()
for url in final_urls_2002:
    try:
        final_df_2002 = final_df_2002.append(getTextFromTribunePage(url,'brides-wanted_2002.csv','grooms-wanted_2002.csv'),ignore_index = True)
    except:
        continue

In [561]:
final_df_2002[300:]# There are several missing values, which occur because the 
                    #number of ads for men and women are not equal and can be cleaned, and 
                    #should not be an issue if a csv file is used for further analysis

,Brides-wanted,Grooms-wanted
300,NaN,Medico/nonMedico/professionally qualified wel...
301,NaN,Wanted educated professional Jat Sikh boy for ...
302,NaN,Alliance invited from NRI vegetarian preferab...
303,NaN,Well settled Jat Sikh parents invite correspon...
304,NaN,Jat Khatri Sikh Doc/S/W/Engr match for Jat Sik...
...,...,...
8478,Wanted well educated smart girl from reputed f...,NaN
8479,Suitable beautiful tall professionally qualifi...,NaN
8480,Delhi based small Saraswat Punjabi Bhardwaj Br...,NaN
8481,Match for Brahmin handsome boy 26/58 BA D a...,NaN


I tried creating a general spider function, but the structure of the website has changed that the tags have changed and I had to wwrite another spider for getting data for 2014.

In [462]:
#Starting with level 0
base_url = 'https://www.tribuneindia.com/2014/forms/archive.htm'
final_urls_2014 = []
url_request = requests.get(base_url)
web_page_soup = bs4.BeautifulSoup(url_request.text,'html.parser')
tagLinks = web_page_soup.findAll('a', href=re.compile('2014'), class_=False)
final_urls = set()
# getting all relevant links and this is the next level
for aTag in tagLinks:
        relurl = aTag.get('href')
       
        level_1_request = requests.get(relurl)
        level_1_soup = bs4.BeautifulSoup(level_1_request.text, 'html.parser')
        level_1_tagLinks = level_1_soup.body.findAll('area',href = re.compile('class'), class_=False)
         # going further down another level and getting all relevant links for the next level 
        # and creating final set of urls
        for a1Tag in level_1_tagLinks:
            relurl = a1Tag.get('href')
            level_2_url = 'https://www.tribuneindia.com/2014'+re.sub(r'^..','',a1Tag.get('href'))
            final_urls_2014.append(level_2_url)

In [556]:
# getting all info for 2014 and storing in dataframe
final_df_2014 = pandas.DataFrame()
for url in final_urls_2014:
    try:
        print(url)
        final_df_2014 = final_df_2014.append(getTextFromTribunePage(url,'brides-wanted_2014.csv','grooms-wanted_2014.csv'),ignore_index = True)
    except:
        continue


https://www.tribuneindia.com/2014/20141026/class-m.htm
https://www.tribuneindia.com/2014/20141026/class-f.htm
https://www.tribuneindia.com/2014/20141026/class-f.htm#2
https://www.tribuneindia.com/2014/20141102/class-m.htm
https://www.tribuneindia.com/2014/20141102/class-f.htm
https://www.tribuneindia.com/2014/20141102/class-f.htm#2
https://www.tribuneindia.com/2014/20141102/class-m.htm
https://www.tribuneindia.com/2014/20141102/class-f.htm
https://www.tribuneindia.com/2014/20141102/class-f.htm#2
https://www.tribuneindia.com/2014/20141109/class-m.htm
https://www.tribuneindia.com/2014/20141109/class-f.htm
https://www.tribuneindia.com/2014/20141109/class-f.htm#2
https://www.tribuneindia.com/2014/20141116/class-m.htm
https://www.tribuneindia.com/2014/20141116/class-f.htm
https://www.tribuneindia.com/2014/20141116/class-f.htm#2
https://www.tribuneindia.com/2014/20141123/class-m.htm
https://www.tribuneindia.com/2014/20141123/class-f.htm
https://www.tribuneindia.com/2014/20141123/class-f.htm#

https://www.tribuneindia.com/2014ass-m.htm
https://www.tribuneindia.com/2014ass-f.htm
https://www.tribuneindia.com/2014ass-f.htm#2
https://www.tribuneindia.com/2014/20140615/class-m.htm
https://www.tribuneindia.com/2014/20140615/class-f.htm
https://www.tribuneindia.com/2014/20140615/class-f.htm#2
https://www.tribuneindia.com/2014/20140615/class-m.htm
https://www.tribuneindia.com/2014/20140615/class-f.htm
https://www.tribuneindia.com/2014/20140615/class-f.htm#2
https://www.tribuneindia.com/2014/20140615/class-m.htm
https://www.tribuneindia.com/2014/20140615/class-f.htm
https://www.tribuneindia.com/2014/20140615/class-f.htm#2
https://www.tribuneindia.com/2014/20140622/class-m.htm
https://www.tribuneindia.com/2014/20140622/class-f.htm
https://www.tribuneindia.com/2014/20140622/class-f.htm#2
https://www.tribuneindia.com/2014/20140622/class-m.htm
https://www.tribuneindia.com/2014/20140622/class-f.htm
https://www.tribuneindia.com/2014/20140622/class-f.htm#2
https://www.tribuneindia.com/2014/

https://www.tribuneindia.com/2014ass-f.htm
https://www.tribuneindia.com/2014ass-f.htm#2
https://www.tribuneindia.com/2014ass-m.htm
https://www.tribuneindia.com/2014ass-f.htm
https://www.tribuneindia.com/2014ass-f.htm#2
https://www.tribuneindia.com/2014/20140126/class-m.htm
https://www.tribuneindia.com/2014/20140126/class-f.htm
https://www.tribuneindia.com/2014/20140126/class-f.htm#2
https://www.tribuneindia.com/2014ass-m.htm
https://www.tribuneindia.com/2014ass-f.htm
https://www.tribuneindia.com/2014ass-f.htm#2
https://www.tribuneindia.com/2014/20140202/class-m.htm
https://www.tribuneindia.com/2014/20140202/class-f.htm
https://www.tribuneindia.com/2014/20140202/class-f.htm#2
https://www.tribuneindia.com/2014/20140202/class-m.htm
https://www.tribuneindia.com/2014/20140202/class-f.htm
https://www.tribuneindia.com/2014/20140202/class-f.htm#2
https://www.tribuneindia.com/2014/20140202/class-m.htm
https://www.tribuneindia.com/2014/20140202/class-f.htm
https://www.tribuneindia.com/2014/20140

https://www.tribuneindia.com/2014/20140119/class-f.htm#2
https://www.tribuneindia.com/2014ass-m.htm
https://www.tribuneindia.com/2014ass-f.htm
https://www.tribuneindia.com/2014ass-f.htm#2
https://www.tribuneindia.com/2014/20140126/class-m.htm
https://www.tribuneindia.com/2014/20140126/class-f.htm
https://www.tribuneindia.com/2014/20140126/class-f.htm#2
https://www.tribuneindia.com/2014/20140126/class-m.htm
https://www.tribuneindia.com/2014/20140126/class-f.htm
https://www.tribuneindia.com/2014/20140126/class-f.htm#2
https://www.tribuneindia.com/2014/20140126/class-m.htm
https://www.tribuneindia.com/2014/20140126/class-f.htm
https://www.tribuneindia.com/2014/20140126/class-f.htm#2
https://www.tribuneindia.com/2014/20140126/class-m.htm
https://www.tribuneindia.com/2014/20140126/class-f.htm
https://www.tribuneindia.com/2014/20140126/class-f.htm#2


In [594]:
final_df_2014['Grooms-wanted'][100:] #will have similar problems of missing values but can be 
                                    #used for analysis if it's a csv file

100      Saini (Pabla) beautiful fair   5'1 KV backgrou...
101      Professionally qualified suitable match for be...
102      Suitable match for fair beautiful Hindu Punjab...
103      Prof qulified boy for MCA MPhil fair slim beau...
104      Punjab Doaba Karnana Parhars educated smart gi...
                               ...                        
26581    LAND FOR SALELand 2 5 10 25 acres on and aroun...
26582    Land 2 5 10 25 acres on and around LandranKhar...
26583                                                     
26584                                                     
26585                                                     
Name: Grooms-wanted, Length: 26486, dtype: object

# <font color="red">Exercise 3</font>
<font color="red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</font>


For this exercise, I extract data from a pdf about internet censorship 'https://geography.fullerton.edu/taylor/550/internetcensorship.pdf'. In the last step, I select a few pages whose data might be relevant to me, and organize the page wise data in a data frame. 

In [47]:
information_extraction_pdf = 'https://geography.fullerton.edu/taylor/550/internetcensorship.pdf'
infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.3
%verypdf.com
532 0 obj
<<
/Linearized 1
/O 534
/H [ 1056413 143    ]
/L 1155272        
/E 1056413        
/N 23
/T 1144512        
>>
endobj
xref
532 44
0000000022 00000 n
0001055770 00000 n
0001055928 00000 n
0001056413 00000 n
0001056557 00000 n
0001056880 00000 n
0001061662 00000 n
0001062097 00000 n
0001062150 00000 n
0001062560 00000 n
0001080168 00000 n
0001080207 00000 n
0001082907 00000 n
0001083000 00000 n
0001083497 00000 n
0001083813 00000 n
0001083998 00000 n
0001087302 00000 n
0001087481 00000 n
0001087551 00000 n
0001087746 00000 n
0001092558 00000 n
0001092847 00000 n
0001093538 00000 n
0001093651 00000 n
0001093856 00000 n
0001105624 00000 n
0001105932 00000 n
0001106889 00000 n
0001107083 00000 n
0001107289 00000 n
0001117753 00000 n
0001118189 00000 n
0001118267 00000 n
0001118472 00000 n
0001131986 00000 n
0001132277 00000 n
0001133200 00000 n
0001133407 00000 n
0001143776 00000 n
0001143846 00000 n
0001143983 00000 


In [262]:


def readPDF(pdfFile, pageno):
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set([21,22])
    #countries()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, [pageno], maxpages=maxpages, password=password,caching=caching, check_extractable=True):
       
        interpreter.process_page(page)
        
    device.close()
    returnedString = retstr.getvalue()
    returnedString = returnedString.replace(' \n ','')
    retstr.close()
    
  
    return returnedString

In [263]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

In [270]:

pages, content = [],[]
for page in range(14,23):
    pages.append('page no '+str(page)) 
    content.append(re.sub(r'\n',' ',readPDF(infoExtractionBytes,page)))
    
    

In [271]:
final_df = pandas.DataFrame({'pages':pandas.Series(pages),'content':pandas.Series(content)})

In [272]:
final_df

,pages,content
0,page no 14,GeoJournal newspaper was arrested for posting...
1,page no 15,been shut down a number of times (http://www. ...
2,page no 16,GeoJournal Internet have been more explicit a...
3,page no 17,"(Kreimer 2006). Thus, Congress has mandated th..."
4,page no 18,GeoJournal The only criterion that remains fo...
5,page no 19,"The information technology revolution, however..."
6,page no 20,"GeoJournal Cunningham, K. (2002). Factors inﬂ..."
7,page no 21,"MacKinnon, R. (2009). China’s censorship 2.0: ..."
8,page no 22,"GeoJournal Zook, M. (2005a). The geography of..."


# <font color="red">Exercise 4</font>

<font color="red">In the two cells immediately following, describe a possible project (e.g., it might end up being your final project, but need not be if you are still searching): **WHAT** you will analyze--the texts you will select and the social game, world and actors you intend to learn about through your analysis (<100 words); **WHY** you will analyze these texts to learn about that context--justify the rationale behind your proposed sample design for this project, based on the readings. What is the social game, social work, or social actors about whom you are seeking to make inferences? What are the virtues of your proposed sample with respect to your research questions? What are its limitations? What are alternatives? What would be a reasonable path to "scale up" your sample for further analysis (i.e., high-profile publication)? (<150 words)? [**Note**: your individual or collective project will change over the course of the quarter as new data and/or analysis opportunities arise or if old ones fade away.] 



## ***What?*** 
<100 words

I am currently interested in understanding how matrimonial advertisements— advertisements that are published seeking matches with prospective brides and grooms— in newspapers have changed over the years, in northern India.  I plan on looking at archived digital ads from 1998 to 2014 with ‘the Tribune’ the largest selling English language paper in 6 major north Indian states. Some of these ads mention traits such as facial complexion, willingness to migrate, caste etc. and I want to see how the characteristics of the social game of seeking partners has changed, and how this might align with developments in and around the states.

## ***Why?***
<150 words

The tradition of seeking matrimony through advertisements has been prevalent in India for several decades. Since these advertisements elicit alliances and aim to act as a filter for prospective partners, they tend to be very descriptive and can be a good way of understanding several social dynamics such as age at which men and women get married, social desirability of a caste, importance of education in matrimony etc, and looking at them can give a great understanding of the social fabric of these states and how it has changed in tandem with other developments. Even though there are several websites that now post these ads, I stick to newspapers since I want to study how things have changed over time, from a point when there were no websites. I believe the limitations for this could be lack of representativeness of the sample, sparse nature of the data, and nuanced language used. This study can be scaled up if it is combined with the e-advertisements and multiple newspapers, to have a more comprehensive understanding of the social landscape.